In [110]:
# Import python modules
import numpy as np
import kaggle
import math
from sklearn.metrics import accuracy_score
from sklearn.linear_model.ridge import Ridge


# Read in train and test synthetic data
def read_synthetic_data():
	print('Reading synthetic data ...')
	train_x = np.loadtxt('../../Data/Synthetic/data_train.txt', delimiter = ',', dtype=float)
	train_y = np.loadtxt('../../Data/Synthetic/label_train.txt', delimiter = ',', dtype=float)
	test_x = np.loadtxt('../../Data/Synthetic/data_test.txt', delimiter = ',', dtype=float)
	test_y = np.loadtxt('../../Data/Synthetic/label_test.txt', delimiter = ',', dtype=float)

	return (train_x, train_y, test_x, test_y)


# Compute MSE
def compute_MSE(y, y_hat):
	# mean squared error
	return np.mean(np.power(y - y_hat, 2))

train_x, train_y, test_x, test_y = read_synthetic_data()
print('Train=', train_x.shape)
print('Test=', test_x.shape)


Reading synthetic data ...
Train= (200,)
Test= (200,)


In [111]:
#Apply polynomial basis expansion
def my_kernel(X,Y,p):
    K = np.zeros((X.shape[0],Y.shape[0]))
    for i,x in enumerate(X):
        for j,y in enumerate(Y):
            K[i,j] = (1+x*y)**p
    return K
    
krrs_polypredictions = []
for deg in [1,2,4,6]:
    #print(my_kernel(train_x.T,train_x,i))
    alpha = np.linalg.inv(my_kernel(train_x.T,train_x,deg) + (0.1*np.eye(train_x.shape[0]))).dot(train_y.reshape(-1,1))
    #print(alpha)
    pred_y = np.zeros((test_x.shape[0],1))
    for i in range(test_x.shape[0]):
        psum = 0
        for j in  range(train_x.shape[0]):
            psum += alpha[j]*(1 + np.dot(test_x[i].T, train_x[j]))**deg
        pred_y[i][0] = psum
    krrs_polypredictions.append(pred_y)
    
#print(krrs_polypredictions)

for lst in krrs_polypredictions:
    print(compute_MSE(np.array(lst).reshape(200,1), test_y.reshape(200,1)))

krrs_trigpredictions = []
for deg in [3,6,10]:
    alpha = np.linalg.inv(trig_kernel(train_x.T,train_x,deg) + (0.1*np.eye(train_x.shape[0]))).dot(train_y)
    pred_y = np.zeros((test_x.shape[0],1))
    #print(alpha)
    for i in range(test_x.shape[0]):
        psum = 0
        for j in  range(train_x.shape[0]):
            tsum = 0
            d = 0.5
            for de in range(deg):
                tsum += np.dot(np.sin(de*d*test_x[i]),np.sin(de*d*train_x[j])) + np.dot(np.cos(de*d*test_x[i]),np.cos(de*d*train_x[j]))
            psum += alpha[j]*(1 + tsum)
        pred_y[i][0] = psum
    krrs_trigpredictions.append(pred_y)

#print(krrs_trigpredictions)

for lst in krrs_trigpredictions:
    print(compute_MSE(np.array(lst).reshape(200,1), test_y.reshape(200,1)))


0.582340444821
0.536822781304
0.441506852453
0.102524426496
0.165771469732
0.118826043104
0.0973803176496


In [114]:
berr_polypredictions = []
for deg in [1,2,4,6]:
    train_x_new = []
    for i in range(train_x.shape[0]):
        train_x_new.append([train_x[i]**j for j in range(deg+1)])
    test_x_new = []
    for i in range(test_x.shape[0]):
        test_x_new.append([test_x[i]**j for j in range(deg+1)])
    clf = Ridge()
    clf.fit(train_x_new, train_y)
    pred_y = clf.predict(test_x_new)
    #print(pred_y)
    berr_polypredictions.append(pred_y)

#print(berr_polypredictions)
for lst in berr_polypredictions:
    print(compute_MSE(np.array(lst).reshape(200,1), test_y.reshape(200,1)))

def trig_expansion(train_x, degree):
    res = [1]
    d = 0.5
    for de in range(degree):
        res.append(np.sin(de*d*train_x))
        res.append(np.cos(de*d*train_x))
    return res

berr_trigpredictions = []
for degree in [3, 5, 10]:
    train_x_new = []
    for i in range(train_x.shape[0]):
        train_x_new.append(trig_expansion(train_x[i],degree))
    test_x_new = []
    for i in range(test_x.shape[0]):
        test_x_new.append(trig_expansion(test_x[i],degree))

    clf = Ridge()
    clf.fit(train_x_new, train_y)
    pred_y = clf.predict(test_x_new)
    berr_trigpredictions.append(pred_y)

#print(berr_trigpredictions)  
for lst in berr_trigpredictions:
    print(compute_MSE(np.array(lst).reshape(200,1), test_y.reshape(200,1)))

0.582447550275
0.536850611277
0.441917855053
0.126113336564
0.166136360724
0.130578295205
0.097819516656


In [150]:
import matplotlib.pyplot as plt
plt.rc('font',size=7)
f, array = plt.subplots(4, 2, figsize=(10,8))
array[0,0].scatter(test_x, test_y, c='b', marker = '*', s=8)
array[0,0].scatter(test_x, krrs_polypredictions[1], c='r', marker = 'o', s=7)
array[0,0].set_xlabel("Test X")
array[0,0].set_ylabel("True/Predicted Y")
array[0,0].set_title("KRRS, Polynomial, lambda = 0.1, degree = 2")

array[0,1].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[0,1].scatter(test_x, berr_polypredictions[1], c='r', marker = 'o', s=7)
array[0,1].set_xlabel("Test X")
array[0,1].set_ylabel("True/Predicted Y") 
array[0,1].set_title("BERR, Polynomial, lambda = 0.1, degree = 2")

array[1,0].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[1,0].scatter(test_x, krrs_polypredictions[3], c='r', marker = 'o', s=7)
array[1,0].set_xlabel("Test X")
array[1,0].set_ylabel("True/Predicted Y")
array[1,0].set_title("KRRS, Polynomial, lambda = 0.1, degree = 6")

array[1,1].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[1,1].scatter(test_x, berr_polypredictions[3], c='r', marker = 'o', s=7)
array[1,1].set_xlabel("Test X")
array[1,1].set_ylabel("True/Predicted Y")
array[1,1].set_title("BERR, Polynomial, lambda = 0.1, degree = 6")

array[2,0].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[2,0].scatter(test_x, krrs_trigpredictions[1], c='r', marker = 'o', s=7)
array[2,0].set_xlabel("Test X")
array[2,0].set_ylabel("True/Predicted Y")
array[2,0].set_title("KRRS, Trignometric, lambda = 0.1, degree = 5")

array[2,1].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[2,1].scatter(test_x, berr_trigpredictions[1], c='r', marker = 'o', s=7)
array[2,1].set_xlabel("Test X")
array[2,1].set_ylabel("True/Predicted Y")
array[2,1].set_title("BERR, Trignometric, lambda = 0.1, degree = 5")

array[3,0].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[3,0].scatter(test_x, krrs_trigpredictions[2], c='r', marker = 'o', s=7)
array[3,0].set_xlabel("Test X")
array[3,0].set_ylabel("True/Predicted Y")
array[3,0].set_title("KRRS, Trignometric, lambda = 0.1, degree = 10")

array[3,1].scatter(test_x, test_y, c='b', marker = '*', s=7)
array[3,1].scatter(test_x, berr_trigpredictions[2], c='r', marker = 'o', s=7)
array[3,1].set_xlabel("Test X")
array[3,1].set_ylabel("True/Predicted Y")
array[3,1].set_title("BERR, Trignometric, lambda = 0.1, degree = 10")

plt.tight_layout()

plt.savefig('../Figures/Plot.png')
plt.close()